In [ ]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check
import duckdb

# Define o caminho para o banco de dados DuckDB
db_path = 'D:\\GitHub\\projeto_Analytics_Engineering\\data_quality\\silver_2.duckdb'

In [8]:
# Definindo o esquema para a tabela "listings_silver"
listings_schema = DataFrameSchema({
    "id": Column(pa.Int64, checks=Check.greater_than(0), nullable=False),  # ID não nulo e maior que 0
    "neighbourhood_cleansed": Column(pa.String, nullable=False),             # Bairro não pode ser nulo
    "price": Column(pa.Float, nullable=False),                              # Preço não nulo
    "number_of_reviews": Column(pa.Int32, checks=Check.greater_than_or_equal_to(0), nullable=False),  # Número de avaliações não nulo
    "review_scores_rating": Column(pa.Float, checks=Check.in_range(0, 5), nullable=False)                 # Avaliação não nula e entre 0 e 5
})

In [9]:
# Conectando ao banco de dados DuckDB especificado
conn = duckdb.connect(database=db_path, read_only=False)

# Carregando os dados da tabela "listings_silver"
df_listings = conn.execute("SELECT * FROM listings_silver").fetchdf()

# Validando o DataFrame com Pandera
try:
    validated_listings_df = listings_schema.validate(df_listings)
    print("Dados validados com sucesso:")
    print(validated_listings_df)
except pa.errors.SchemaError as e:
    print("Erro na validação dos dados:", e)
finally:
    conn.close()  # Fechando a conexão com o banco de dados

Dados validados com sucesso:
                        id neighbourhood_cleansed  price  number_of_reviews  \
0                    17878             Copacabana  310.0                319   
1                    25026             Copacabana  203.0                291   
2                   220377                 Tijuca  220.0                  5   
3                    35764             Copacabana  201.0                476   
4                   223073                Ipanema  321.0                471   
...                    ...                    ...    ...                ...   
52684  1186981497123032289        Barra da Tijuca  630.0                  0   
52685  1187078788620039832             Copacabana  103.0                  0   
52686  1184700608791969952                 Tijuca  150.0                  0   
52687  1184818035513976907                 Catete  307.0                  0   
52688  1184844634544738969             Copacabana  311.0                  0   

       review_scores_r